# Semantic LLM caching

_**NOTE:** this uses Cassandra's experimental "Vector Similarity Search" capability.
At the moment, this is obtained by building and running an early alpha from a specific branch of the codebase._

The Cassandra-backed "semantic cache" for prompt responses is imported like this:

In [1]:
from langchain.cache import CassandraSemanticCache

As usual, a database connection is needed to access Cassandra. The following assumes
that a _vector-search-capable Cassandra cluster_ is running locally. Adjust as needed.

In [2]:
from cqlsession import getLocalSession, getLocalKeyspace
localSession = getLocalSession()
localKeyspace = getLocalKeyspace()

An embedding function and an LLM are needed.

Below is the logic to instantiate the LLM and embeddings of choice. We choose to leave it in the notebooks for clarity.

In [3]:
from llm_choice import suggestLLMProvider

llmProvider = suggestLLMProvider()
# (Alternatively set llmProvider to 'VertexAI', 'OpenAI' ... manually if you have credentials)

if llmProvider == 'VertexAI':
    from langchain.llms import VertexAI
    from langchain.embeddings import VertexAIEmbeddings
    llm = VertexAI()
    myEmbedding = VertexAIEmbeddings()
    print('LLM+embeddings from VertexAI')
elif llmProvider == 'OpenAI':
    from langchain.llms import OpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)
    myEmbedding = OpenAIEmbeddings()
    print('LLM+embeddings from OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM+embeddings from OpenAI


**Note**: for the time being you have to explicitly _turn on this experimental flag_ on the `cassio` side:

In [4]:
import cassio
cassio.globals.enableExperimentalVectorSearch()

## Create the cache

At this point you can instantiate the semantic cache.

_Note: in the following it is made clear, through the `table_name_prefix` parameter, that different embeddings will require separate tables. This is done here to avoid mismatches when running this demo over and over with varying embedding functions: in most applications, where a single embedding suffices, there's no need to be this finicky._

In [5]:
cassSemanticCache = CassandraSemanticCache(
    session=localSession,
    keyspace=localKeyspace,
    embedding=myEmbedding,
    table_name_prefix=f'semantic_cache_{llmProvider}_',
)

Make sure the cache starts empty with:

In [6]:
cassSemanticCache.clear_through_llm(llm=llm)

Configure the cache at a LangChain global level:

In [7]:
import langchain
langchain.llm_cache = cassSemanticCache

## Use the cache

Now try submitting a few prompts to the LLM and pay attention to the response times.

If the LLM is actually run, they should be the order of a few seconds; but in case of a cache hit, it will be way less than a second.

Notice that you get a cache hit even after rephrasing the question.

In [8]:
%%time
SPIDER_QUESTION_FORM_1 = "How many eyes do spiders have?"
# A new question should take long
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 6.12 ms, sys: 4.38 ms, total: 10.5 ms
Wall time: 1.67 s


'\n\nMost spiders have eight eyes, but some have fewer.'

In [9]:
%%time
# Second time, very same question, this should be quick
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 7.32 ms, sys: 0 ns, total: 7.32 ms
Wall time: 104 ms


'\n\nMost spiders have eight eyes, but some have fewer.'

In [10]:
%%time
SPIDER_QUESTION_FORM_2 = "How many eyes does a spider generally have?"
# Just a rephrasing: but it's the same question,
# so it will just take the time to evaluate embeddings
llm(SPIDER_QUESTION_FORM_2)

CPU times: user 9.37 ms, sys: 163 µs, total: 9.53 ms
Wall time: 296 ms


'\n\nMost spiders have eight eyes, but some have fewer.'

Time for a really new question:

In [11]:
%%time
LOGIC_QUESTION_FORM_1 = "Is absence of proof the same as proof of absence?"
# A totally new question
llm(LOGIC_QUESTION_FORM_1)

CPU times: user 13.3 ms, sys: 0 ns, total: 13.3 ms
Wall time: 1.55 s


'\n\nNo, absence of proof is not the same as proof of absence.'

In [12]:
%%time
SPIDER_QUESTION_FORM_3 = "How many eyes are on the head of a typical spider?"
# Trying to catch the cache off-guard :)
llm(SPIDER_QUESTION_FORM_3)

CPU times: user 13.5 ms, sys: 1.06 ms, total: 14.6 ms
Wall time: 416 ms


'\n\nMost spiders have eight eyes, but some have fewer.'

In [13]:
%%time
LOGIC_QUESTION_FORM_2 = "Is it true that the absence of a proof equates the proof of an absence?"
# Switching to the other question again
llm(LOGIC_QUESTION_FORM_2)

CPU times: user 9.44 ms, sys: 0 ns, total: 9.44 ms
Wall time: 715 ms


'\n\nNo, absence of proof is not the same as proof of absence.'

## Additional options

When creating the semantic cache, you can specify a few other options such as the metric used to calculate the similarity and the number of entries to retrieve in the ANN step (i.e. those on which the exact requested metric is computed for the final filtering). Here is an example which uses the L2 metric:

In [14]:
anotherCassSemanticCache = CassandraSemanticCache(
    session=localSession,
    keyspace=localKeyspace,
    embedding=myEmbedding,
    distance_metric='l2',
    score_threshold=0.4,
    num_rows_to_fetch=12,
    table_name_prefix=f'semantic_cache_{llmProvider}_',
)

This cache builds on the same database table as the previous one, as can be seen e.g. with:

In [15]:
lookup = anotherCassSemanticCache.lookup_with_id_through_llm(
    LOGIC_QUESTION_FORM_2,
    llm,
)
if lookup:
    docId, response = lookup
    print(docId)
    print(response)
else:
    print('No match.')

77add13036bcaa23c74ebf2ab2c56441
[Generation(text='\n\nNo, absence of proof is not the same as proof of absence.', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n\nNo, absence of proof is not the same as proof of absence.', generation_info={'finish_reason': 'stop', 'logprobs': None})]


## Stale entry control

### Time-To-Live (TTL)

You can configure a time-to-live property of the cache, with the effect of automatic eviction of cached entries after a certain time.

Setting `langchain.llm_cache` to the following will have the effect that entries vanish in an hour:

In [16]:
cacheWithTTL = CassandraSemanticCache(
    session=localSession,
    keyspace=localKeyspace,
    embedding=myEmbedding,
    ttl_seconds=15,
    table_name_prefix=f'semantic_cache_{llmProvider}_',
)

### Manual cache eviction

Alternatively, you can invalidate individual entries one at a time, just like you saw for the exact-match `CassandraCache` cache.

But this is an index based on sentence similarity, so this time the procedure has two steps: **first**, a lookup to find the _id_ of the matching document:

In [17]:
lookup = cassSemanticCache.lookup_with_id_through_llm(SPIDER_QUESTION_FORM_1, llm)
if lookup:
    docId, response = lookup
    print(docId)
else:
    print('No match.')

0a1339bc659790da078a4352c05bf422


_you can see that querying for another form for the "same" question will result in the same id:_

In [18]:
lookup2 = cassSemanticCache.lookup_with_id_through_llm(SPIDER_QUESTION_FORM_2, llm)
if lookup:
    docId2, response2 = lookup2
    print(docId2)
else:
    print('No match.')

0a1339bc659790da078a4352c05bf422


and **second**, the document id is used in the actual cache eviction (again, you have to additionally provide the LLM):

In [19]:
cassSemanticCache.delete_by_document_id_through_llm(docId, llm)

As a check, try asking that question again:

In [20]:
%%time
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 7.27 ms, sys: 2.24 ms, total: 9.51 ms
Wall time: 1.08 s


'\n\nMost spiders have eight eyes, but some have six or fewer.'

### Whole-cache deletion

Lastly, as you have seen earlier, you can empty the cache entirely, for a given LLM, with:

In [21]:
cassSemanticCache.clear_through_llm(llm=llm)